# Load rankings and categories

In [1]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import json,os,pdb
import tqdm
import warnings

# Choose the universe

Select the number of coins to consider. Right now we are still using a market cap criteria

In [2]:
N = 50

# Load data


In [4]:
df = pd.read_parquet('output/panels/daily_20221004.parquet.gzip') # modify the path according to your needs
df.columns = [c.split('.')[-1].replace('_','') for c in df.columns]
df.index = df.index.set_levels([df.index.levels[0], pd.to_datetime(df.index.levels[1])])

Load the listing file and check for the rankings

In [6]:
with open('output/listingmaps/listing_map_20221004.json',encoding='utf-8') as json_data:
    data = json.load(json_data)
listmap = pd.DataFrame(data['data'])
listmap.sort_values('cmc_rank',inplace=True)
listmap.set_index('cmc_rank',inplace=True)
listmap['symbol_id'] = listmap['symbol']+ '_'+ listmap['id'].astype(str)

# Filter by the universe
Select only the symbols that interest you. According to our previous choice of N, in this case we are filtering for the top 50 by market cap

In [7]:
symbols = list(listmap.loc[:N,'symbol_id'].values)

Remove stablecoins by using the category JSON (available in sakai as the listing file)

In [7]:
with open('output/categories/category_stablecoin.json',encoding='utf-8') as json_data:
    data = json.load(json_data)
stable_df = pd.json_normalize(data['data']['coins'],max_level=5)
stable_df['symbol_id'] = stable_df['symbol'] + '_' + stable_df['id'].astype(str)
stablecoins = list(stable_df['symbol_id'].values)

In [8]:
# get the top 200 by mkt cap
symbols = set(listmap.loc[:N,'symbol_id'].values) - set(stablecoins)

for l in stablecoins:
    if l in symbols:
        print('Stablecoins in the list!')